# A Tour of Digital Audio in Python/Numpy
### <a href = "https://www.ctralie.com">Chris Tralie</a> 

## Bio

### Upper Dublin Computer Club President 2005-2007

<img src = "Pics/acsl2007.jpg" width="50%">

### Graduated Upper Dublin 2007

<img src = "Pics/udgrad.jpg" width="50%">

### Princeton Class of 2011, Major in Electrical Engineering + Computer Science

<img src = "Pics/princeton.jpg" width="20%">

### Duke University Ph.D. in Electrical And Computer Engineering, 2017

<img src = "Pics/duke.jpg" width="50%">

### Assistant Professor in Math/CS At Ursinus College Since Fall 2019

<img src = "Pics/seraiah3.jpg" width="50%">

# Digital Audio: The Basics

I'll be giving some highlights of a course I'm currently teaching to Ursinus undergraduates.  Course web site at <a href = "https://ursinus-cs372-s2023.github.io/CoursePage/">this link</a>


In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import IPython.display as ipd
from dsp import *

First, what is audio?  Let's load some in and find out!

In [ ]:
sr, x = wavfile.read("Audio/femalecountdown.wav")
x = x/32768
plt.figure()
plt.plot(x)
ipd.Audio(x, rate=sr)

In [ ]:
# Show sample rate

In [ ]:
# Show slicing

In [ ]:
# Show reversing

### <a href = "https://ursinus-cs372-s2023.github.io/CoursePage/ClassExercises/Week1/Week1_AudioReverseGame/">Audio reversing game</a>

In [ ]:
# Show noise

## Filtering Audio

In [ ]:
# Windowed energy
def get_energy(x, win):
    """
    Compute the energy in a window around each sample
    
    Parameters
    ----------
    x: ndarray(N)
        Audio samples
    win: int
        Window length in which to sum energy
    
    Returns
    -------
    ndarray(N): Energy arond each sample
    """
    eng = x**2
    w = np.zeros(win//2)
    eng = np.concatenate((w, eng, w))
    eng = np.cumsum(eng)
    return eng[win::] - eng[0:-win]


sr, x = wavfile.read("Audio/femalecountdown.wav")
x = x/32768
eng = get_energy(x, 1000)

plt.figure()
plt.subplot(211)
plt.plot(x)
plt.ylabel("Audio Waveform")
plt.subplot(212)
plt.plot(eng)
plt.xlabel("Sample Index")
plt.ylabel("Energy")
plt.tight_layout()

In [ ]:
# Filter by energy

In [ ]:
# Windowed zero crossings
def get_zcs(x, win):
    """
    Compute the zero crossings in a window around each sample
    
    Parameters
    ----------
    x: ndarray(N)
        Audio samples
    win: int
        Window length in which to sum energy
    
    Returns
    -------
    ndarray(N): Zero crossing count arond each sample
    """
    zcs = np.sign(x[1::]) - np.sign(x[0:-1]) == 2
    w = np.zeros(win//2)
    zcs = np.concatenate((w, [0], zcs, w))
    zcs = np.cumsum(zcs)
    return zcs[win::] - zcs[0:-win]

sr, x = wavfile.read("Audio/femalecountdown.wav")
x = x/32768
zcs = get_zcs(x, 4000)

plt.figure()
plt.subplot(211)
plt.plot(x)
plt.ylabel("Audio Waveform")
plt.subplot(212)
plt.plot(eng)
plt.xlabel("Sample Index")
plt.ylabel("Zero Crossings")
plt.tight_layout()

In [ ]:
# Filter by zero crossings


## Periodic Sounds/Pitches

Define period, define frequency, and show cosine

In [ ]:
# Show cosine sampled at a particular period

# What do we notice when the frequency goes up?

Equation for pitch perception

In [ ]:
# Make happy birthday tune with cosines

In [ ]:
# Make happy birthday tune with square waves

# Convolution / Echoes

<img src = "Convolution.gif">

In [ ]:
sr, x = wavfile.read("Audio/jessiesgirl.wav")
x = x/32768
ipd.Audio(x, rate=sr)

In [ ]:
# Show example with JFK Tunnel / Jessie's Girl
sr, h = wavfile.read("Audio/jfktunnel.wav")
h = h/32768
ipd.Audio(h, rate=sr)

In [ ]:
from scipy.signal import fftconvolve

# Convolve x and h, compare result to x

### Comb Filter




In [ ]:
# Example on noise

In [ ]:
# Example on audio
sr, x = wavfile.read("Audio/robot.wav")
ipd.Audio(x, rate=sr)

# Spectrograms 


### <a href = "https://musiclab.chromeexperiments.com/spectrogram">Live spectrogram demo</a>


In [ ]:
sr, x = wavfile.read("Audio/violincastinets.wav")
x = x/32768
win = 2048
hop = 512
S = stft(x, win, hop)
plt.figure()
plot_stft(S, sr, hop)
plt.ylim([0, 8000])
ipd.Audio(x, rate=sr)

### Beepy Tunes, aka Hacking Shazam

In [ ]:
sr, x = wavfile.read("Audio/rr.wav")

In [ ]:
from scipy import ndimage

win = 2048
hop = 1024
time_win = 8
freq_win = 5
max_freq = 256

S = np.abs(stft(x, win, hop))
M = ndimage.maximum_filter(S, size=(freq_win*2+1, time_win*2+1)) == S
M = M[0:256, :]
I, J = np.meshgrid(np.arange(M.shape[0]), np.arange(M.shape[1]), indexing='ij')
I = I[M == 1]
J = J[M == 1]

plt.figure()
plt.imshow(amplitude_to_db(S)[0:max_freq, :], cmap='gray', aspect='auto')
plt.scatter(J, I, s=1, c='C1')
plt.ylim([0, max_freq])

In [ ]:
N = x.size
t = np.arange(win)/sr
y = np.zeros(N)
hann_win = 0.5*(1 - np.cos(2*np.pi*np.arange(win)/win))
for i, j in zip(I, J):
    freq = i*sr/win
    blip = np.cos(2*np.pi*freq*t)
    j1 = j*hop
    L = len(y[j1:j1+win])
    y[j1:j1+win] += (hann_win*blip)[0:L]

In [ ]:
ipd.Audio(y, rate=sr)

# FM Synthesis (Time Permitting)

In [ ]:
sr = 44100
t = np.arange(sr*2)/sr
fc = 440
fm = 440
I = 4
y = np.cos(2*np.pi*fc*t + I*np.sin(2*np.pi*fm*t))
ipd.Audio(y, rate=sr)